<a href="https://colab.research.google.com/github/DeepHMS/voila-barplot-tool/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Widgets ---
file_uploader = widgets.FileUpload(
    accept='.csv', multiple=False, description='Upload CSV'
)

x_dropdown = widgets.Dropdown(description='X Column')
y_dropdown = widgets.Dropdown(description='Y Column')

color_picker = widgets.ColorPicker(value='blue', description='Bar Color')
title_text = widgets.Text(value='My Bar Plot', description='Title')
xlabel_text = widgets.Text(value='X-axis', description='X Label')
ylabel_text = widgets.Text(value='Y-axis', description='Y Label')

dpi_slider = widgets.IntSlider(value=100, min=50, max=300, step=10, description='DPI')

plot_button = widgets.Button(description='Draw Plot', button_style='success')
download_button = widgets.Button(description='Download', button_style='info')

output = widgets.Output()

df = None  # Global variable to store uploaded DataFrame

# --- Functions ---
def on_file_upload(change):
    global df
    if file_uploader.value:
        uploaded_file = next(iter(file_uploader.value.values()))
        content = uploaded_file['content']
        df = pd.read_csv(io.BytesIO(content))

        # Update dropdown options and default values
        x_dropdown.options = df.columns.tolist()
        x_dropdown.value = df.columns[0]
        y_dropdown.options = df.columns.tolist()
        y_dropdown.value = df.columns[1]

        with output:
            clear_output()
            print("CSV uploaded successfully! Select X and Y columns to draw the plot.")
    else:
        with output:
            clear_output()
            print("Please upload a CSV file.")

def draw_plot(_):
    if df is not None and x_dropdown.value and y_dropdown.value:
        with output:
            clear_output()
            fig, ax = plt.subplots(figsize=(8, 5))
            ax.bar(df[x_dropdown.value], df[y_dropdown.value], color=color_picker.value)
            ax.set_title(title_text.value)
            ax.set_xlabel(xlabel_text.value)
            ax.set_ylabel(ylabel_text.value)
            plt.show()
    else:
        with output:
            clear_output()
            print("Please upload a file and select columns.")

def download_plot(_):
    if df is not None and x_dropdown.value and y_dropdown.value:
        buf = io.BytesIO()
        fig, ax = plt.subplots(figsize=(8, 5))
        ax.bar(df[x_dropdown.value], df[y_dropdown.value], color=color_picker.value)
        ax.set_title(title_text.value)
        ax.set_xlabel(xlabel_text.value)
        ax.set_ylabel(ylabel_text.value)
        plt.savefig(buf, format='png', dpi=dpi_slider.value)
        buf.seek(0)

        # Save locally on server
        with open("bar_plot.png", "wb") as f:
            f.write(buf.getbuffer())

        with output:
            print("Plot saved as bar_plot.png (download from server).")
    else:
        with output:
            clear_output()
            print("Nothing to download.")

# --- Event Listeners ---
file_uploader.observe(on_file_upload, names='value')
plot_button.on_click(draw_plot)
download_button.on_click(download_plot)

# --- Layout ---
ui = widgets.VBox([
    file_uploader,
    widgets.HBox([x_dropdown, y_dropdown]),
    color_picker,
    title_text, xlabel_text, ylabel_text,
    dpi_slider,
    widgets.HBox([plot_button, download_button]),
    output
])

display(ui)
